In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_tam
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'tam_train_negative_augmented')
    dev_data = read_task(dataset_location , split = 'tam_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_tam
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['ai4bharat/indic-bert']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Tamil' + 'Augmented' +str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 

FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.742255687713623
Validation loss per 100 evaluation steps: 1.587224829905104
DEV ACC: 0.5560057382133995
DEV Precision: 0.41564687326828625
DEV Recall: 0.42055248704223613
DEV F1Score: 0.39521317699631353
BEST ACCURACY -->  DEV: 0.61924
BEST PRECISION -->  DEV: 0.43946
BEST RECAL

...

FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.5579514503479004
Validation loss per 100 evaluation steps: 1.6968799977019282
DEV ACC: 0.5672689205955336
DEV Precision: 0.4328913331580197
DEV Recall: 0.4406640563298583
DEV F1Score: 0.41372237519719607
BEST ACCURACY -->  DEV: 0.6193
BEST PRECISION -->  DEV: 0.47048
BEST RECALL -->  DEV: 0.4603
BEST F1SCORE -->  DEV: 0.43668
TIME PER EPOCH: 28.162663793563844

Training epoch: 10
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED B

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 0.78549724817276
Validation l

FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.4237174987792969
Validation loss per 100 evaluation steps: 1.766624910406547
DEV ACC: 0.5721735421836228
DEV Precision: 0.40792139129496646
DEV Recall: 0.40359400315646604
DEV F1Score: 0.3844137410499452
BEST ACCURACY -->  DEV: 0.62277
BEST PRECISION -->  DEV: 0.45748
BEST RECALL -->  DEV: 0.4617
BEST F1SCORE -->  DEV: 0.43173
TIME PER EPOCH: 28.195887450377146

In [6]:
print(best_dev_acc)

0.6227706265508685


In [7]:
print(best_epoch)

4


In [8]:
print(best_dev_precision)

0.45747667791735463


In [9]:
print(best_dev_recall)

0.4616977957017311


In [10]:
print(best_dev_f1score)

0.43173408453395484


In [11]:
print(overall_list_dev_acc)

[[0.5834754962779156, 0.6077659739454094, 0.6192424007444168, 0.6067191377171216, 0.5927807071960298, 0.5873332816377171, 0.5827776054590571, 0.5733366935483871, 0.5560057382133995, 0.5661639267990074], [0.5969098945409429, 0.6200566066997518, 0.6145897952853597, 0.6121859491315137, 0.6059049317617865, 0.5934204404466501, 0.5831459367245658, 0.5813236662531017, 0.5808196339950372, 0.5562771401985112], [0.5959599875930521, 0.6163926799007444, 0.6217044044665013, 0.6105575372208436, 0.5933041253101736, 0.6064089640198511, 0.5875077543424317, 0.5572658188585607, 0.579559553349876, 0.5650201612903226], [0.6008452233250621, 0.6193005583126551, 0.6141439205955336, 0.6036755583126551, 0.6138531327543425, 0.6034235421836228, 0.5810910359801489, 0.5709910049627791, 0.5672689205955336, 0.5904156327543425], [0.5884382754342432, 0.6039469602977667, 0.6227706265508685, 0.619630117866005, 0.5824674317617865, 0.578939205955335, 0.5816338399503722, 0.5652721774193549, 0.5721735421836228, 0.56992478287

In [12]:
print(overall_list_dev_precision)

[[0.41620691059935905, 0.39431555553368164, 0.43223449643514206, 0.43034747151169667, 0.4290255700298299, 0.4188873719864393, 0.4394645192187396, 0.4271273292603256, 0.41564687326828625, 0.4243434914211092], [0.35351479755853354, 0.4308946594918275, 0.44286252732767634, 0.43566759790516346, 0.4510080325198947, 0.4435173774050301, 0.4496290844828071, 0.4403559025551997, 0.44188549066325633, 0.4194521298666025], [0.2993418027115163, 0.45997062523425475, 0.46426686643235127, 0.4668502786682438, 0.42592192459753825, 0.45356158438737937, 0.42592078528658955, 0.41776557643128365, 0.43305801910872493, 0.4250551338899542], [0.41426759691844, 0.43093388582270564, 0.4586018152711296, 0.45068354487180373, 0.4704838090637343, 0.46457280581089444, 0.4477468485003119, 0.42785316486709596, 0.4328913331580197, 0.4206790003385622], [0.33400191456525663, 0.45747667791735463, 0.454984875844549, 0.45241882026949753, 0.4470259257992468, 0.4165574191100735, 0.4285200328635633, 0.4118427375704235, 0.40792139

In [13]:
print(overall_list_dev_recall)

[[0.3635947649499689, 0.3385760638929916, 0.36757991108220855, 0.44473059832276146, 0.4354013522253552, 0.4218002565950111, 0.44067347516165273, 0.4078229860301081, 0.42055248704223613, 0.41449103001093307], [0.3188234144412166, 0.3713193382675786, 0.4059084861429284, 0.4347150217344317, 0.42455655172906365, 0.4358534778735555, 0.4318340906477116, 0.4423964709454953, 0.42774895176077327, 0.4170658996330069], [0.29121394820798774, 0.4275421821532339, 0.3915499138327985, 0.42663958281531056, 0.4424905420390357, 0.40342097705552715, 0.3924280445818954, 0.41017281211198564, 0.4135496338691228, 0.4105219834276537], [0.36141901122210746, 0.39220904417175617, 0.425710375696784, 0.460295545541962, 0.4596625982255958, 0.43409608525198634, 0.44659361322419755, 0.4302655093369108, 0.4406640563298583, 0.40373119416589426], [0.3511213148511142, 0.4259537650662591, 0.40687496965761377, 0.4295074519946105, 0.4616977957017311, 0.44080849162429375, 0.3983619848681931, 0.4193788134204016, 0.403594003156

In [14]:
print(overall_list_dev_f1score)

[[0.35638732607657725, 0.32467359670629486, 0.3600513127719307, 0.40995282525184745, 0.40407295614205846, 0.39640189217531285, 0.41531000826369674, 0.39645139407260677, 0.39521317699631353, 0.3957243534257395], [0.3012014411804219, 0.36177405129235807, 0.3885142384184353, 0.4058767627762948, 0.41190393088149346, 0.4158351000775335, 0.4160642653150318, 0.4168050794511596, 0.4126178172442753, 0.39880855784612157], [0.2601033468675601, 0.41899157261597175, 0.3868103247722207, 0.4220168853315848, 0.4064137051829618, 0.3940258071831762, 0.38101577215229554, 0.39352911023945264, 0.39788283324635154, 0.3969349264075154], [0.3561650769915082, 0.38121275093838247, 0.4088170292521979, 0.42618934483241533, 0.43667749110349774, 0.42245278028135064, 0.42054050612394683, 0.4076316083157751, 0.41372237519719607, 0.39036805535763475], [0.3242847570153722, 0.4014257904743172, 0.3955523036449553, 0.4078940944203661, 0.43173408453395484, 0.3994967607811968, 0.38909594314811025, 0.39398698914510555, 0.384

In [15]:
#The best model is 